In [0]:
%run "/Workspace/Users/ruchika.b.mhetre@v4c.ai/vstone_project/vstone_databricks_pipeline/src/notebooks/00_Setup/project_config"

In [0]:
# Run this in your Ingestion Notebook
#%run "../00_Setup/project_config"
import re

cat_csv_path = f"{volume_path}/catalogs.csv"

# 1. Read with the correct delimiter (sep=";")
df_cat_raw = (spark.read.format("csv")
              .option("header", "true")
              .option("sep", ";") # This is the fix!
              .option("inferSchema", "true")
              .load(cat_csv_path))

# 2. Sanitize column names to remove spaces and special characters
def clean_column_name(name):
    return re.sub(r'[ ,;{}()\n\t=]+', '_', name).strip('_')

df_cat_clean = df_cat_raw.toDF(*[clean_column_name(c) for c in df_cat_raw.columns])

# 3. Overwrite the broken table
df_cat_clean.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalog_name}.{schema_name}.catalogs_bronze")

print("✅ Catalogs table fixed! New Columns:")
print(df_cat_clean.columns)

In [0]:
# Run this in your 02_Bronze_Ingestion folder
#%run "../00_Setup/project_config"
import re

cat_csv_path = f"{volume_path}/catalogs.csv"

# 1. Read with the correct semicolon separator
df_cat_raw = (spark.read.format("csv")
              .option("header", "true")
              .option("sep", ";") # The critical fix
              .option("inferSchema", "true")
              .load(cat_csv_path))

# 2. Sanitize column names (removes spaces/dots for Delta compatibility)
def clean_col(name):
    return re.sub(r'[ ,;{}()\n\t=]+', '_', name).strip('_')

df_cat_clean = df_cat_raw.toDF(*[clean_col(c) for c in df_cat_raw.columns])

# 3. Overwrite the catalog table
df_cat_clean.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalog_name}.{schema_name}.catalogs_bronze")

print("✅ Catalogs table fixed! Individual columns are now available.")

In [0]:
# Drop the existing broken table to clear metadata
spark.sql(f"DROP TABLE IF EXISTS {catalog_name}.{schema_name}.catalogs_bronze")

# Re-ingest properly
df_cat_fixed = (spark.read.format("csv")
                .option("header", "true")
                .option("sep", ";") 
                .load(f"{volume_path}/catalogs.csv"))

# Sanitize and Save
import re
clean_cols = [re.sub(r'[ ,;{}()\n\t=]+', '_', c).strip('_') for c in df_cat_fixed.columns]
df_cat_fixed.toDF(*clean_cols).write.format("delta").saveAsTable(f"{catalog_name}.{schema_name}.catalogs_bronze")